## Spam Classifier
__Machine Learning & Data Science Assignment by Thöni Andreas__  
2024

---
Click [here](https://github.com/ajayhanssen/spam_filter_ml), to get to the Github repository

---

#### Preprocessing Infrastructure
This section introduces the __EmailPreprocessor__ class, which is able to convert a raw email file into a stemmed and normalized string. It inherits from sklearn's BaseEstimator and TransformerMixin classes, so that it is possible to use the class with sklearn's __make_pipeline()__ later.

In [110]:
from email import policy
from email.parser import BytesParser
from nltk.stem.snowball import SnowballStemmer
import re
from string import punctuation
from sklearn.base import BaseEstimator, TransformerMixin

class EmailPreprocessor(BaseEstimator, TransformerMixin):
    """
    class for processing emails, extracting email parts and applying regex and stemming
    """

    def __init__(self) -> None:
        pass

    def extract_email_from_file(self, file_path : str) -> str:
        """
        Extracts the subject, sender, recipient and body of an email file
        arguments: file_path - path to the email file
        return: string containing the email
        """

        # Open the email file
        with open(file_path, 'rb') as file:
            # Parse the email using the default policy
            email_message = BytesParser(policy=policy.default).parse(file)
        
        # Extract headers, like subject, sender and recipient
        # the .get() method is used to avoid errors if the header is not present (returns failobj instead)
        subject = email_message.get('Subject', '(No Subject)')
        sender = email_message.get('From', '(Unknown Sender)')
        recipient = email_message.get('To', '(Unknown Recipient)')
        
        # Extract body
        # initilaize empty string for storing the body
        body = ""
        if email_message.get_body(preferencelist=('plain', 'html')):
            body_content = email_message.get_body(preferencelist=('plain', 'html'))
            body = body_content.get_content()  # Automatically decodes and returns the content
        
        # return email as string
        return f"{subject} {sender} {recipient} {body.strip()}"

    def stem_and_regex(self, email_string : str) -> str:
        """
        Applies stemming and regex operations to the email
        arguments: email - dictionary with the email parts or string with the email
        return: string with the processed email
        """

        # convert to lowercase
        email_string = email_string.lower()

        ## perform regex operations
        # change email adresses to 'emailaddr'
        email_string = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', 'emailaddr', email_string)

        # change urls to 'httpaddr'
        email_string = re.sub(r'(http|https)://[^\s]*', 'httpaddr', email_string)

        # change time to 'time'
        email_string = re.sub(r'\b\d{1,2}:\d{1,2}(:\d{1,2})?\b', 'time', email_string)

        # change date to 'date'
        email_string = re.sub(r'\b\d{1,2}/\d{1,2}/\d{4}\b', 'date', email_string)

        # change dollar to 'dollar'
        email_string = re.sub(r'\$\S+', 'dollar', email_string)

        # change www-URLs to 'wwwaddr'
        email_string = re.sub(r'\bwww\.[^\s]*\b', 'wwwaddr', email_string)

        # change percentages to 'percent'
        email_string = re.sub(r'\b\d+%', 'percent', email_string)

        # change ip to 'ipaddr'
        email_string = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', 'ipaddr', email_string)

        # change numbers to 'number'
        email_string = re.sub(r'\b\d+\b', 'number', email_string)

        # Remove itemization prefixes (bullets, numbered lists, etc.)
        email_string = re.sub(r'^\s*[\d\w][\.\)\-]\s*|[\u2022\u2219\u25CB·]\s*', '', email_string, flags=re.MULTILINE)

        ## other processing tasks
        # remove punctuation
        email_string = email_string.translate(str.maketrans('', '', punctuation))

        # remove newlines and tabs
        email_string = email_string.replace('\n', ' ')
        email_string = email_string.replace('\t', ' ')

        # remove multiple spaces
        email_string = re.sub(r'\s+', ' ', email_string)

        ## stemming
        # stem the words using a Snowball stemmer (newer version of Porter stemmer apparently)
        stemmer = SnowballStemmer('english')

        # iterate through the words in the email-string and apply stemming, then join them back together
        email_string = ' '.join([stemmer.stem(word) for word in email_string.split()])

        # return single string containing the processed email
        return email_string

    def process_email(self, email_file_path : str) -> str:
        """
        Process an email file, calls the extract_email_from_file and stem_and_regex functions
        arguments: email_file_path - path to the email file
        return: string with the processed email
        """

        # call the extract_email_from_file function to get the email parts from the file
        email_parts = self.extract_email_from_file(email_file_path)

        # call the stem_and_regex function to process the email parts
        processed = self.stem_and_regex(email_parts)

        # return the processed email
        return processed
    
    ## methods for the transformer (needed to work with sklearn pipelines)
    def fit(self, X, y=None) -> None:
        """
        Fit method for the transformer, does nothing in this case
        """

        return
    
    def transform(self, X : list[str]) -> list[str]:
        """
        Transform method for the transformer, applies the stem_and_regex function to the emails
        arguments: X - list of emails
        return: list of processed emails
        """

        return [self.stem_and_regex(email) for email in X]


---
#### Load and preprocess ham and spam emails
This section introduces a function which can be used to extract email-files from a given directory, and preprocess them. In this project, only the dataset from 2002 was used.

In [111]:
# If this is set to false, only the easy_ham folders will be used
# If this is set to true, the hard_ham directory is also used
full_dataset = True

In [112]:
import os


def load_data_from_directories(directory : str, label : str) -> tuple[list[str], list[int]]:
    """
    Load email data from given directories (ham and spam)
    arguments: directory - path to the file directory
               label - 'ham' or 'spam' label
    return: emails - list of email strings
            labels - list of labels (0 for ham, 1 for spam)
    """

    # Create an instance of the EmailPreprocessor class defined above
    preprocessor = EmailPreprocessor()

    # Initialize lists to store the emails and labels
    emails = []
    labels = []

    ## Load ham emails
    # iterate through the files in the ham directory
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)

        # as the open() function sometimes throwed an error caused by encoding issues,
        # these parts are wrapped in a try-except block (happens 3 times in the code)
        try:
            # process the email file using the process_email method of the EmailPreprocessor class
            email = preprocessor.process_email(file_path)
            # append the processed email to the emails list
            emails.append(email)

            if label == 'ham':
                # append the label (0 for ham) to the labels list
                labels.append(0)
            elif label == 'spam':
                # append the label (1 for spam) to the labels list
                labels.append(1)
            else:
                # if the label is not 'ham' or 'spam', raise a ValueError
                raise ValueError("Label must be 'ham' or 'spam'")

        except:
            # if an error ocurred while reading from the file, print an error message
            print(f"Error processing file {file_path}")

    # return the emails and labels lists
    return emails, labels


## use the function to load the data
# define the paths to the ham and spam directories
easy_ham_dir = "./datasets/20021010_easy_ham/easy_ham"
hard_ham_dir = "./datasets/20021010_hard_ham/hard_ham"
spam_dir = "./datasets/20021010_spam/spam"

# call the function to load and preprocess the data
emails, labels = load_data_from_directories(easy_ham_dir, 'ham')
emails_spam, labels_spam = load_data_from_directories(spam_dir, 'spam')

# if the full_dataset flag is set to True, load the hard_ham directory too
if full_dataset:
    emails_hard_ham, labels_hard_ham = load_data_from_directories(hard_ham_dir, 'ham')

    # concatenate the hard_ham emails and labels to the easy_ham ones
    emails += emails_hard_ham
    labels += labels_hard_ham

# concatenate the spam emails and labels as well
emails += emails_spam
labels += labels_spam

# print the number of emails loaded
print(f"Number of emails loaded: {len(emails)}")

Error processing file ./datasets/20021010_spam/spam\0226.409b6577c79d85773d50cb37fde4ba79
Error processing file ./datasets/20021010_spam/spam\0329.5c22249fa35fff050675e7df4433b89f
Error processing file ./datasets/20021010_spam/spam\0399.b9eab4251d9263129290cf7fc2aa4c7a
Number of emails loaded: 3299


In [113]:
# print the first email, just to check if everything is working
print(emails[0])

re new sequenc window robert elz emailaddr chris garrigu emailaddr date wed number aug number time number from chris garrigu emailaddr messageid emailaddr i cant reproduc this error for me it is veri repeat like everi time without fail this is the debug log of the pick happen time pickit exec pick inbox list lbrace lbrace subject ftp rbrace rbrace numbernumb sequenc mercuri time exec pick inbox list lbrace lbrace subject ftp rbrace rbrace numbernumb sequenc mercuri time ftocpickmsg number hit time mark number hit time tkerror syntax error in express int note if i run the pick command by hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace numbernumb sequenc mercuri number hit that where the number hit come from obvious the version of nmh im use is delta pick version pick nmhnumbernumbernumb compil on fuchsiacsmuozau at sun mar number time ict number and the relev part of my mhprofil delta mhparam pick seq sel list sinc the pick command work the sequenc actual both of them

---
#### Vectorize the data using a TF-IDF Vectorizer
A TF-IDF Vectorizer is able to convert a collection of text objects into numerical vectors, which made it excellent for this use-case. TF-IDF stands for __Term Frequency-Inverse Document Frequency__ and evaluates how important a certain word is to a single instance in a collection of objects.  
If a words appears often in one instance, it is considered important at first, but if it also appears in many other instances, its importance is decreased.

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create a TfidfVectorizer object with the desired parameters
# stop_words='english' uses the built-in list of English stop words and removes them
# max_features=2500 limits the number of features to the 2500 most important words
vectorizer = TfidfVectorizer(stop_words="english", max_features=2500)

# fit the vectorizer to the emails and transform them into a matrix
X = vectorizer.fit_transform(emails).toarray()
y = labels

# X is now a matrix, with rows representing the emails and columns the features
print(X.shape)

(3299, 2500)


---
#### Splitting into training and test set
In this section, the classic split in to the training and test set is conducted. The distribution used is 80% to 20%.

In [115]:
from sklearn.model_selection import train_test_split

# split the data into training and testing sets (80% training, 20% testing, classic 42 random state)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

---
#### Trying different classifiers
In the following sections, diffferent classifiers are used and evaluated using the following metrics:

In [116]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

__Naive Bayes model__ (is apparently often used for spam classifiers)

In [117]:
from sklearn.naive_bayes import MultinomialNB

mnb_model = MultinomialNB()
mnb_model.fit(X_train, y_train)

y_pred = mnb_model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")

Accuracy: 0.9348484848484848
Precision: 0.76
Recall: 0.8
F1 Score: 0.7794871794871795


__Support Vector Classifier__

In [118]:
from sklearn.svm import SVC

svc_model = SVC()
svc_model.fit(X_train, y_train)

y_pred = svc_model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")

Accuracy: 0.9833333333333333
Precision: 0.9666666666666667
Recall: 0.9157894736842105
F1 Score: 0.9405405405405406


__Random Forest Classifier__

In [119]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")

Accuracy: 0.9757575757575757
Precision: 0.9876543209876543
Recall: 0.8421052631578947
F1 Score: 0.9090909090909091


---
#### Transforming new data to use with the models
In this short section, it is shown how the functions and classes implemented before could be used in a pipeline to classify new, incoming emails

In [129]:
from sklearn.pipeline import make_pipeline

# create a pipeline with an EmailPreprocessor and the TfidfVectorizer that was fitted before
pipeline = make_pipeline(EmailPreprocessor(), vectorizer)


# define new emails
some_new_spam = "Hello, I am a Nigerian prince and I would like to offer you a million dollars. Please send me your bank account details."
some_new_ham = "Hi, I was wondering if i could get my Nintendo Switch back. I only lent it to you for a week and it's been a month now."

# transform the new emails using the pipeline
X_new = pipeline.transform([some_new_spam, some_new_ham])

# convert sparse matrix to dense (SVC was trained on dense matrix, TF-IDF outputs a sparse vector, needs to be converted)
X_new = X_new.toarray()

# predict the label of the new emails using the model
labels = svc_model.predict(X_new)

# the model correctly classifies the emails as spam ([1]) and ham ([0])	
print(labels)

[1 0]


---
## Classification results

The length of the feature vector (being controlled by changing the _max_features_ parameter of the TF-IDF Vectorizer) had significant impact on the performance of the model. When using only the easy ham emails, a value of 1000 was deemed best, when training on the full dataset (containing the hard ham as well), a value of 2500 yielded the best results.

__Easy ham only:__ (using a feature vector with a length of 1000)

| __Classifier__     | __Accuracy__ | __Precision__ | __Recall__ | __F1-Score |
|--------------------|--------------|---------------|------------|------------|
| Multinomial NB     | 0.9803       | 1.0           | 0.8762     | 0.9341     |
| __SVM Classifier__ | __0.9869__   | __1.0__       | __0.9175__ | __0.9570__ |
| Random Forest      | 0.9803       | 0.9885        | 0.8866     | 0.9348     |


__Full Dataset:__ (using a feature vector with a length of 2500)

| __Classifier__     | __Accuracy__ | __Precision__ | __Recall__ | __F1-Score |
|--------------------|--------------|---------------|------------|------------|
| Multinomial NB     | 0.9318       | 0.75          | 0.7895     | 0.7692     |
| __SVM Classifier__ | __0.9833__   | __0.9667__    | __0.9158__ | __0.9405__ |
| Random Forest      | 0.9758       | 0.9759        | 0.8526     | 0.9101     |

As can be seen in the tables above, a __SVM classifier__ led to the __best__ results in both occations compared to the Multinomial Naive-Bayes and the Random-Forest classifier.

---